In [2]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


from sklearn.linear_model import Ridge, Lasso, ElasticNet, SGDRegressor

%matplotlib inline

In [3]:
# Gathering Data
df = pd.read_csv("insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
# Data Cleaning
# The total rows is 1338 and 7 cloumn
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
# Checking Missing values
df.isnull().sum()
# there is no missing values

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [6]:
# Spliting the dataset
#805 of the data into the training set and 20% into the testing set
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['charges']), df['charges'],
                                                   train_size = 0.8, random_state = 620)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(1070, 6) (1070,)
(268, 6) (268,)


In [7]:
# Preprocessing

scaler = StandardScaler()
encoder = OneHotEncoder(drop = 'first')
preprocessor = ColumnTransformer([('scaler', scaler, make_column_selector(dtype_include='number')),
                                  ('encoder', encoder, make_column_selector(dtype_include='object'))])

processed_x_train = preprocessor.fit_transform(x_train)
processed_x_test = preprocessor.transform(x_test)

In [8]:
# Model selection with Cross-Validation
for model in [Ridge(), Lasso(), ElasticNet(), SGDRegressor()]:
    cv_score = cross_val_score(model, processed_x_train, y_train, scoring = 'neg_mean_squared_error', cv=5)
    print("Accuracy score for %s: %.1f"%(model.__class__.__name__, -cv_score.mean()))

Accuracy score for Ridge: 36947911.2
Accuracy score for Lasso: 36941490.3
Accuracy score for ElasticNet: 90603216.3
Accuracy score for SGDRegressor: 36926977.2


In [9]:
# Tune parameters with Grid Search CV

# Creating an array of numbers from 0 to 1 in steps of 0.05
lasso_param_grid = {'alpha': np.arange(0, 1, 0.05)}

lasso_tune = GridSearchCV(Lasso(), lasso_param_grid, cv=5,
                          scoring = 'neg_mean_squared_error',
                          return_train_score = True, n_jobs=1,
                          verbose=1)
lasso_tune.fit(processed_x_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.526e+10, tolerance: 1.246e+07 Linear regression models with null weight for the l1 regularization term are more ef

GridSearchCV(cv=5, estimator=Lasso(), n_jobs=1,
             param_grid={'alpha': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=1)